In [52]:
from tensorflow import keras
from tensorflow.keras import layers
import splitfolders
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import keras_tuner as kt
import scipy
import warnings
import os

#### Image Parameters

In [2]:
batch_size = 32
img_height = 256
img_width = 256
img_channels = 3

#### Data

In [44]:
train_split_test_dir='../0. Project Data/Reshaped zero-padded and split data'
train_ds = image_dataset_from_directory(
    train_split_test_dir + '/train',
    image_size=(img_width, img_height),
    batch_size=batch_size)
val_ds = image_dataset_from_directory(
    train_split_test_dir + '/val',
    image_size=(img_width, img_height),
    batch_size=batch_size)
test_ds = image_dataset_from_directory(
    train_split_test_dir + '/test',
    image_size=(img_width, img_height),
    batch_size=batch_size)

Found 2678 files belonging to 4 classes.
Found 667 files belonging to 4 classes.
Found 843 files belonging to 4 classes.


In [5]:
def build_model(hp):
    conv_base = keras.applications.vgg16.VGG16(
        weights='imagenet',
        include_top=False)
    conv_base.trainable = False
    
    data_augmentation = keras.Sequential([layers.RandomFlip('horizontal'), 
                                          layers.RandomRotation(0.1), 
                                          layers.RandomZoom(0.2),])
    
    hp_units = hp.Int('units', min_value=128, max_value=512, step=32)
    hp_dropout = hp.Float('dropout', min_value=.1, max_value=.5, step=.2)
    
    inputs = keras.Input(shape=(img_height, img_width, img_channels))
    x = data_augmentation(inputs)
    x = keras.applications.vgg16.preprocess_input(x)
    x = conv_base(x)
    x = layers.Flatten()(x)
    x = layers.Dense(hp_units)(x)
    x = layers.Dropout(hp_dropout)(x)
    outputs = layers.Dense(4, activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model 

In [6]:
tuner=kt.BayesianOptimization(build_model,
                             objective='val_accuracy',
                             max_trials = 100,
                             executions_per_trial=1,
                             directory='kt_logs',
                             overwrite=False); # if changed tuner, set to True

In [7]:
tuner.search_space_summary(extended=True)

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
dropout (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.2, 'sampling': 'linear'}


In [8]:
callbacks = [
 keras.callbacks.EarlyStopping(monitor="val_loss", patience=7),
]

based off previous transfer learning model over fitting very early, we only do 20 epochs

In [9]:
tf.get_logger().setLevel('ERROR')

In [10]:
tuner.search(train_ds,
             batch_size=batch_size,
             epochs=20,
             validation_data=val_ds,
             callbacks=callbacks,
             verbose=2,
)

Trial 100 Complete [00h 10m 00s]
val_accuracy: 0.9490255117416382

Best val_accuracy So Far: 0.9640179872512817
Total elapsed time: 16h 13m 54s


In [29]:
best_hps=tuner.get_best_hyperparameters(num_trials=5)

In [30]:
print(f'''
The hyperparameter search is complete. The optimal number of units in the first densely-connected layer is\n
{best_hps[0].get('units')}\n
and the optimal dropout rate for the dropout layer is \n
{best_hps[0].get('dropout')}.
''')


The hyperparameter search is complete. The optimal number of units in the first densely-connected layer is

128

and the optimal dropout rate for the dropout layer is 

0.5.



In [21]:
tuner.results_summary(num_trials=5)

Results summary
Results in kt_logs\untitled_project
Showing 5 best trials
Objective(name="val_accuracy", direction="max")

Trial 063 summary
Hyperparameters:
units: 128
dropout: 0.5
Score: 0.9640179872512817

Trial 048 summary
Hyperparameters:
units: 160
dropout: 0.5
Score: 0.9595202207565308

Trial 075 summary
Hyperparameters:
units: 288
dropout: 0.30000000000000004
Score: 0.9580209851264954

Trial 086 summary
Hyperparameters:
units: 128
dropout: 0.1
Score: 0.95652174949646

Trial 059 summary
Hyperparameters:
units: 128
dropout: 0.5
Score: 0.9550225138664246


In [42]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
    keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', patience=10)
    ]
    history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    batch_size=batch_size,
    callbacks=callbacks)
    val_loss_per_epoch = history.history['val_loss']
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [41]:
combined_train_and_val_ds = tf.data.Dataset.concatenate(train_ds, val_ds).shuffle(buffer_size=100)

In [46]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    model.fit(
        combined_train_and_val_ds,
        batch_size=batch_size, epochs=int(best_epoch * 1.25)) #train for more epoch cause more data
    return model

In [47]:
best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(test_ds)
    best_models.append(model)

Epoch 1/100
84/84 [==============================] - 30s 338ms/step - loss: 17.6870 - accuracy: 0.8208 - val_loss: 9.5534 - val_accuracy: 0.9235
Epoch 2/100
84/84 [==============================] - 28s 333ms/step - loss: 10.1162 - accuracy: 0.9089 - val_loss: 8.3331 - val_accuracy: 0.9280
Epoch 3/100
84/84 [==============================] - 28s 334ms/step - loss: 8.9658 - accuracy: 0.9115 - val_loss: 7.5784 - val_accuracy: 0.9295
Epoch 4/100
84/84 [==============================] - 28s 337ms/step - loss: 6.5853 - accuracy: 0.9302 - val_loss: 9.5049 - val_accuracy: 0.9220
Epoch 5/100
84/84 [==============================] - 28s 338ms/step - loss: 4.7697 - accuracy: 0.9358 - val_loss: 10.0272 - val_accuracy: 0.9160
Epoch 6/100
84/84 [==============================] - 28s 338ms/step - loss: 4.7884 - accuracy: 0.9414 - val_loss: 6.3597 - val_accuracy: 0.9355
Epoch 7/100
84/84 [==============================] - 28s 335ms/step - loss: 3.6166 - accuracy: 0.9485 - val_loss: 7.0027 - val_accura

105/105 [==============================] - 32s 294ms/step - loss: 1.4603 - accuracy: 0.9584
Epoch 30/37
105/105 [==============================] - 32s 294ms/step - loss: 1.4406 - accuracy: 0.9698
Epoch 31/37
105/105 [==============================] - 32s 294ms/step - loss: 1.5421 - accuracy: 0.9647
Epoch 32/37
105/105 [==============================] - 32s 294ms/step - loss: 0.8436 - accuracy: 0.9740
Epoch 33/37
105/105 [==============================] - 32s 293ms/step - loss: 1.0220 - accuracy: 0.9695
Epoch 34/37
105/105 [==============================] - 32s 293ms/step - loss: 1.1948 - accuracy: 0.9749
Epoch 35/37
105/105 [==============================] - 32s 295ms/step - loss: 1.2536 - accuracy: 0.9686
Epoch 36/37
105/105 [==============================] - 32s 297ms/step - loss: 1.4025 - accuracy: 0.9659
Epoch 37/37
27/27 [==============================] - 4s 141ms/step - loss: 3.9888 - accuracy: 0.9193
Epoch 1/100
84/84 [==============================] - 30s 337ms/step - loss: 27.

Epoch 2/100
84/84 [==============================] - 28s 337ms/step - loss: 9.7927 - accuracy: 0.9111 - val_loss: 8.3471 - val_accuracy: 0.9145
Epoch 3/100
84/84 [==============================] - 28s 339ms/step - loss: 5.2954 - accuracy: 0.9302 - val_loss: 4.9512 - val_accuracy: 0.9355
Epoch 4/100
84/84 [==============================] - 28s 336ms/step - loss: 3.5593 - accuracy: 0.9451 - val_loss: 7.2014 - val_accuracy: 0.9190
Epoch 5/100
84/84 [==============================] - 28s 337ms/step - loss: 3.3813 - accuracy: 0.9451 - val_loss: 6.3393 - val_accuracy: 0.9205
Epoch 6/100
84/84 [==============================] - 28s 335ms/step - loss: 1.8364 - accuracy: 0.9533 - val_loss: 5.7066 - val_accuracy: 0.9280
Epoch 7/100
84/84 [==============================] - 28s 334ms/step - loss: 1.8497 - accuracy: 0.9529 - val_loss: 4.4249 - val_accuracy: 0.9340
Epoch 8/100
84/84 [==============================] - 28s 337ms/step - loss: 1.2649 - accuracy: 0.9664 - val_loss: 4.0729 - val_accuracy:

In [48]:
best_models

In [50]:
best_models[0].predict(test_ds)

27/27 [==============================] - 4s 144ms/step


array([[1.0000000e+00, 0.0000000e+00, 6.2287876e-15, 0.0000000e+00],
       [1.7366094e-35, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 4.1981965e-21, 0.0000000e+00],
       ...,
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 2.0871519e-25, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00]],
      dtype=float32)

In [54]:
for index, hp_mod in enumerate(best_models):
    if not os.path.exists('./saved_models'):
            os.makedirs('./saved_models')
    hp_mod.save('./saved_models/my_model'+ str(index+1))

In [56]:
new_model = tf.keras.models.load_model('saved_models/my_model1')

In [57]:
new_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 sequential_3 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 tf.__operators__.getitem_3   (None, 256, 256, 3)      0         
 (SlicingOpLambda)                                               
                                                                 
 tf.nn.bias_add_3 (TFOpLambd  (None, 256, 256, 3)      0         
 a)                                                              
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 flatten_3 (Flatten)         (None, 32768)             0   

In [58]:
new_model.predict(test_ds)

27/27 [==============================] - 4s 146ms/step


array([[1.0067170e-08, 1.9183634e-18, 1.0000000e+00, 0.0000000e+00],
       [4.4414654e-33, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00],
       [9.9985695e-01, 0.0000000e+00, 1.4298240e-04, 0.0000000e+00],
       ...,
       [1.0000000e+00, 0.0000000e+00, 2.0871519e-25, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00]],
      dtype=float32)

In [59]:
new_model.evaluate(test_ds)

27/27 [==============================] - 4s 142ms/step - loss: 3.0054 - accuracy: 0.9359


[3.0053701400756836, 0.935943067073822]

include validation data as part of final training with chosen hyperparameters

find best epoch

retrain on best epoch+validation data extend epoch by a little more cause more data

use test data